In [103]:
#This book will take the historic data available on morning star for the big 4 and apply the CER model to it.
#It will calculate the amount to buy at the start of the next trading at 7:30pm each night.
#Outputting this to a googlesheet and calculating how much cash is is currently holding based on the close values.
#Assumptions made: There are no transaction fees, dividends do not exist, we can trade at the open 
#instantly guaranteeing the price at last nights close, all shares must be priced in the same currency,
#it is the year 2019 (otherwise bank holidays might now apply).

#We will be rebalancing once a day.

#This will run 4 portfollios, the big 4, then three portfollios randomly picked by three people
#and all data will be stored at "https://docs.google.com/spreadsheets/d/19zkugdOI8EHTyWjjWgYmR5Ip8bXuVsaAHZtVgWfcMnU/edit?usp=sharing"

from lxml import html
import requests
import datetime
import numpy as np
import pandas as pd
from math import log

#We first start with a function that will take a morning star historic pricing url and turn the historic data into a panda dataframe.

def PriceList(URL):
    #This function takes a url from morning star histroical prices page of a share
    from lxml import html
    import requests
    import datetime
    import numpy as np
    import pandas as pd
    from math import log

    html=requests.get(URL).content
    df_list=pd.read_html(html)


    lis1=df_list[-4]
    l=len(lis1)
    modlis1=lis1.drop(l-1)


    lis2=df_list[-3]
    l=len(lis2)
    modlis2=lis2.drop(l-1)


    lis3=df_list[-2]
    l=len(lis3)
    modlis3=lis3.drop(l-1)


    bigdata=modlis1.append(modlis2,ignore_index=True)
    bigdata=bigdata.append(modlis3, ignore_index=True)

    pricelist= bigdata.iloc[:,1]
    return pricelist


In [104]:
#We now introduce a function that will take a dataframe of prices and output the sample mean and sample volitility anulised
#on the assumption of 252 trading days a year.

def MaVol(A):
    #For a data frame of prices this calculates the mean and vol of the stock as given by DeGroot 1989
    S=0
    n=len(A)
    for i in range(1,n):
        S=S+log(A.iloc[i-1]/A.iloc[i])

    Mu=S/(n-1)

    Vol=0
    for i in range(1,n):
        Vol=Vol+(log(A.iloc[i-1]/A.iloc[i])-Mu)**2
    
    Vol=Vol/(n-2)
#Note in the mean we have n-1 from n bits of data, as the mean is calculated
#based on change, so we only have n-1 bits of data to calculate from.
#For the varaince we have n-2, as from a sample of y values using 1/y is biased
#whereas 1/(y-1) isn't
    Vol=Vol**0.5
    Scaler=(252/n)**0.5
#This is used to scale our volatility from a sample of 21 days to the 
#annulised volatility

    return [Mu,Vol*Scaler]

In [105]:
#The following function will take price lists for stocks and calculate the covariance between them annulised

def Covar(A,B):
    #Returns the sample covariance between two price lists
    if len(A)==len(B):
        StatsA=MaVol(A)
        StatsB=MaVol(B)
        Cov=0
        for i in range(1,len(A)):
            Cov=Cov+(log(A.iloc[i-1]/A.iloc[i])-StatsA[0])*(log(B.iloc[i-1]/B.iloc[i])-StatsB[0])
        
        Cov=Cov/(len(A)-2)
        Scaler=(252/len(A))
        #Here we have to anualise the covar
        return(Cov*Scaler)
    else:
        #We deal with the case that the website has mismatching lengths for different stocks by
        #shortening the longer one to match the shorter one.
        #Ideally we would have more data, but at the moment I am lacking data and will have to manage
        #with such small sets, leading to inaccurate results.
        l=min(len(A),len(B))
        A=A[:l]
        B=B[:l]
       
        StatsA=MaVol(A)
        StatsB=MaVol(B)
        Cov=0
        for i in range(1,len(A)):
            Cov=Cov+(log(A.iloc[i-1]/A.iloc[i])-StatsA[0])*(log(B.iloc[i-1]/B.iloc[i])-StatsB[0])
        
        Cov=Cov/(len(A)-2)
        Scaler=(252/len(A))
        #Here we have to anualise the covar
        return(Cov*Scaler)


In [106]:
#We now define a function that can give the vector of expected returns from a collection of price lists.
def FinMeanMult(*arg):
    #This function takes in a collection of historic prices of shares
    #from morning star, it then calulcates the expected returns vector
    #matrix of this data
    l=len(arg)
    A=np.array(np.zeros(l))
    for i in range(l):
        MaVol(arg[i])
        
        A[i]=MaVol(arg[i])[0]
    
    return A


In [107]:
#We now calculate  the function for the covariance matrix for a collection of shares
def FinCovMult(*arg):
    #This function takes in a collection of historic prices and evaluates
    #the covariance matrix
    l=len(arg)
    CovMat=np.zeros((l,l))
    for i in range(l):
        for j in range(i,l):
            sig=Covar(arg[i],arg[j])
            CovMat[i,j]=sig
            if i!=j:
                CovMat[j,i]=sig
    return CovMat


In [108]:
#Now following "Portfolio Theory with Matrix Algebra" we create a function that
#will find a variance miminsing portfollio of a given expected return and 
#pricelist
def MinPort(Exp,*args):
    l=len(args)
    b=np.zeros(l+2)
    Mean=FinMeanMult(*args)
    Sig=FinCovMult(*args)

    #We define the vector b as given by page 12
    b[l]=Exp
    b[l+1]=1

    #We now define the A matrix
    A=np.zeros((l+2,l+2))
    for i in range(l):
        for j in range(l):
            A[i,j]=2*Sig[i,j]

    for i in range(l):
        A[l,i]=Mean[i]
        A[i,l]=Mean[i]
        A[l+1,i]=1
        A[i,l+1]=1
    
   
    Port=np.dot(np.linalg.inv(A),b)[:l]
    ER=np.dot(Port,Mean)
    Var=np.dot(Port,np.dot(Sig,Port))
   
    print("Expected return is", ER, "with variance", Var)
    return Port
    
    

    
    
    
    
    
    
    

In [109]:
#We define the following program that will tell us if the following day is a day that banks are open.

def TradeOpenTomorrow():
    #This program determines if banks are open to trade tomorrow in 2019.
    # It returns 0 if banks are closed tomorrow. And 1 if they are open.
    #It does this by checking if tomorrow is a weekend or a bank holiday.
    tomorrow =  datetime.date.today() + datetime.timedelta(days=1)
    d1 = tomorrow.strftime("%d/%m/%Y")
    wkday=tomorrow.weekday()
    BHol2019=["01/01/2019","17/03/2019","19/04/2019","22/04/2019","06/05/2019","27/05/2019","12/07/2019","26/08/2019","25/12/2019","26/12/2019"]
    if wkday in [5,6]:
        return 0
    elif d1 in BHol2019:
        return 0
    else:
        return 1


In [110]:
#To make things as general as possible we now introduce a function that takes given URLs
#and produces a tuple of their price lists.
def PLFromURL(*args):
    B=[]
    for i in range(len(args)):
        B.append(PriceList(args[i]))
    return B


In [111]:
#We will need a function that cantake our current portfollio and evaluate it's value.
def PortValue(*args):
    #This function returns the current value of all holdings.
    P=PLFromURL(*args)
    
    ValueVec=np.ones(len(args)+1)
    for i in range(len(args)):
        ValueVec[i]=P[i][0]
    
    col_len = len(ws.col_values(1))
    vect=np.array(ws.row_values(col_len)[1:len(args)+2])
    
    Amount=[float(i) for i in vect]
    Cash=np.dot(ValueVec,Amount)
    return Cash


In [112]:
#We now introduce a funtion that will update our spreadsheet with the amount of each stock it should
#buy at the start of trading tomorrow and the amount of money currenly held at close.
#This assumes the open and close prices are the same, and that we can instantaneously trade at open.

def SheetUpdate(Exp,*args):
    #We first start by checking that tomorrow's values haven't already been caluclated and that today is a trading day. As this is a one trade per day strat.
    #Otherwise it will just add another row to the spreadsheet.
    col_len = len(ws.col_values(1))
    tomorrow =  datetime.date.today() + datetime.timedelta(days=1)
    tomorrow=tomorrow.strftime("%d/%m/%Y")

    if ws.cell(col_len,1).value!=tomorrow:
        if TradeOpenTomorrow()==1:
            PLs=PLFromURL(*args)
        #We have now generated the price lists
        
            A=MinPort(Exp,*PLs)
            #We have now calculated what we should hold tomorrow.
            
            ValueAtClose=PortValue(*args)
            
            #here we have calculated the portfollio that we should buy at the begining of trading the next day, and the value that 
            #We get from selling our current portfollio.
            
            
            #We now define the value vector
            ValueVec=np.ones(len(args)+1)
            for i in range(len(args)):
                    ValueVec[i]=PLs[i][0]
                    
            C=np.zeros(len(A))
            for i in range(len(A)):
                C[i]=A[i]*ValueAtClose/ValueVec[i]
 
            
            
            B = [tomorrow]
            B.extend(C)
            B.extend([0,ValueAtClose])
            ws.insert_row(B, col_len+1)

    else:
        if TradeOpenTomorrow()==1:
            #we now repeat the above code but such that first we delete the lowest row.
            #This is incase I have run the code several times in a day (probably for testing purposes)
            #But the code should only take it's actual evaluation once a day
            ws.delete_row(col_len)
            col_len = len(ws.col_values(1))
            PLs=PLFromURL(*args)
        #We have now generated the price lists
        
            A=MinPort(Exp,*PLs)
            #We have now calculated what we should hold tomorrow.
            
            ValueAtClose=PortValue(*args)
            
            #here we have calculated the portfollio that we should buy at the begining of trading the next day, and the value that 
            #We get from selling our current portfollio.
            
            
            #We now define the value vector
            ValueVec=np.ones(len(args)+1)
            for i in range(len(args)):
                    ValueVec[i]=PLs[i][0]
                    
            C=np.zeros(len(A))
            for i in range(len(A)):
                C[i]=A[i]*ValueAtClose/ValueVec[i]
 
            
            
            B = [tomorrow]
            B.extend(C)
            B.extend([0,ValueAtClose])
            ws.insert_row(B, col_len+1)

In [113]:
#We now initialize my sheet which contains the data of my previous portfollios in the form
#date held, B shares, H shares, L shares, R shares, Cash 

#This first block of code initalizes the code accessing the google sheet of historic trades and cash value
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)



In [114]:
#Here we define our variables, starting with our shares for the Big 4
ss = client.open('CER')
Bar="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007NZP%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007NZP&ClientFund=0&CurrencyId=BAS"
HSBC="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007OFH%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007OFH&ClientFund=0&CurrencyId=BAS"
Llo="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P000090RG%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000090RG&ClientFund=0&CurrencyId=BAS"
RBS="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P000090MW%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000090MW&ClientFund=0&CurrencyId=BAS"
ws = ss.worksheet("Big4")
SheetUpdate(0.01,Bar,HSBC,Llo,RBS)

Expected return is 0.01 with variance 0.023204856110666167


In [115]:
#We now do Chen's picks

ITV="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007OI2%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007OI2&ClientFund=0&CurrencyId=BAS"
JustEat="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00012MB5%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00012MB5&ClientFund=0&CurrencyId=BAS"
Tesco="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007OYV%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007OYV&ClientFund=0&CurrencyId=BAS"
Vodafone="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007WPO%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007WPO&ClientFund=0&CurrencyId=BAS"
ws=ss.worksheet("Chen")
SheetUpdate(0.12,ITV,JustEat,Tesco,Vodafone)

Expected return is 0.11999999999999998 with variance 4.368468080398069


In [116]:
# We now do Sam's picks
BP="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P000090MI%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000090MI&ClientFund=0&CurrencyId=BAS"
Sainsbury="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P000090N6%5D3%5D0%5DE0WWE%24%24ALL&Id=0P000090N6&ClientFund=0&CurrencyId=BAS"
MS="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007OL2%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007OL2&ClientFund=0&CurrencyId=BAS"
Uni="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007P0W%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007P0W&ClientFund=0&CurrencyId=BAS"
ws=ss.worksheet("Sam")
SheetUpdate(0.03,BP,Sainsbury,MS,Uni)



Expected return is 0.030000000000000006 with variance 0.3767336311909156


In [117]:
#James' picks
Tobacco="http://tools.morningstar.co.uk/uk/stockreport/default.aspx?tab=0&vw=sp&SecurityToken=0P00007O1O%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007O1O&ClientFund=0&CurrencyId=BAS"
ws=ss.worksheet("James")
SheetUpdate(0.045,Tobacco,Sainsbury)

Expected return is 0.044999999999999984 with variance 112.2355781290376
